In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# index = sys.path.index('')
# sys.path.pop(index)
path = sys.path[0]
parent_dir = os.path.abspath(os.path.join(path, os.pardir))

for root, dirs, files in os.walk(parent_dir):
    for dir in dirs:
        sys.path.append(os.path.join(root, dir))

sys.path.append(parent_dir)
sys.path.pop(0)
print(sys.path)

['/usr/lib64/python36.zip', '/usr/lib64/python3.6', '/usr/lib64/python3.6/lib-dynload', '', '/home/ctv.nguyenhvk/.local/lib/python3.6/site-packages', '/usr/local/lib64/python3.6/site-packages', '/usr/local/lib/python3.6/site-packages', '/usr/lib64/python3.6/site-packages', '/usr/lib/python3.6/site-packages', '/home/ctv.nguyenhvk/.local/lib/python3.6/site-packages/IPython/extensions', '/home/ctv.nguyenhvk/.ipython', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/.git', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/experiments', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/output', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/quantum_optimal_control', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/qoc_venv', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/.git/branches', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/.git/hooks', '/home/ctv.nguyenhvk/proj-quantum-opt-control/qutrit-qoc/.git/info', '/home/ctv.nguyen

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import inspect

from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.grape import Grape

data_path = '../output/x01_π/'

In [21]:
from qiskit_ibm_provider import IBMProvider

IBMProvider.save_account('030e0f3b6562aeef01f326920c2b5c95160de76135f801f67cce23704c77a274aa5241b45c7777d8a2bcfb4c71803c8796f4d6896241914aba81fc7281bae508', overwrite=True)

provider = IBMProvider()
backend = provider.get_backend('ibm_lagos')

In [22]:
ns = 1.0e9
config = backend.configuration()
dt = config.dt
acquire_alignment = config.timing_constraints['acquire_alignment']
granularity = config.timing_constraints['granularity']
pulse_alignment = config.timing_constraints['pulse_alignment']
lcm = np.lcm(acquire_alignment, pulse_alignment)

In [23]:
qubit = 0
f01 = backend.properties().qubits[qubit][2].value * 1e9
anhar = backend.properties().qubits[qubit][3].value * 1e9

In [24]:
f01

5235359100.418953

In [25]:
anhar

-339867138.55915606

In [5]:
def get_closest_multiple_of(value, base_number):
    return int(value + base_number/2) - (int(value + base_number/2) % base_number)

def get_closest_multiple_of_16(num):
    return get_closest_multiple_of(num, granularity)

In [3]:
def ket(i):
  if(i == 0):
    return np.array([[1], [0], [0], [0]])
  if(i == 1):
    return np.array([[0], [1], [0], [0]])
  if(i == 2):
    return np.array([[0], [0], [1], [0]])
  if(i == 3):
    return np.array([[0], [0], [0], [1]])

def bra(i):
  if(i == 0):
    return np.array([[1, 0, 0, 0]])
  if(i == 1):
    return np.array([[0, 1, 0, 0]])
  if(i == 2):
    return np.array([[0, 0, 1, 0]])
  if(i == 3):
    return np.array([[0, 0, 0, 1]])

def Pi(i):
  return ket(i)*bra(i)

def sigma_x(i, j):
  return ket(j)@bra(i) + ket(i)@bra(j)

def sigma_y(i, j):
  return 1j*(ket(j)@bra(i) - ket(i)@bra(j))

def sigma_z(i, j):
  return ket(i)@bra(i) - ket(j)@bra(j)

In [4]:
ns = 1.0e9
dt = 2/9/ns
f01 = 5235359100.418953
anhar = -339867138.55915606

In [5]:
#Defining time scales

steps = 144 #1-2: 160
# steps = get_closest_multiple_of_16(steps)
total_time = steps * dt * ns #ns dt = 0.2222222222

# Choose optimizing State transfer or Unitary gate
state_transfer = True

#Defining H0
qubit_state_num = 2 #change
qubit_num = 1 
freq_ge = 0 #5.235350062289255 #GHz #change
g_ops = 0.073036776 #GHz #change
alpha = anhar/ns

# ens = np.array([ 2*np.pi*ii*(freq_ge - 0.5*(ii-1)*alpha) for ii in np.arange(qubit_state_num)])
# ens = np.array([0, 0, alpha, 3*alpha])
# Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
# Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
# Q_z   = np.diag(np.arange(0,qubit_state_num))
# Q_I   = np.identity(qubit_state_num)
# H_q = np.diag(ens)


ens = np.array([2 * np.pi * ii * (freq_ge) for ii in np.arange(qubit_state_num)])
Q_x = np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1) + np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1)
Q_y = (0 + 1j) * (np.diag(np.sqrt(np.arange(1, qubit_state_num)), 1) - np.diag(np.sqrt(np.arange(1, qubit_state_num)), -1))
Q_z = np.diag(np.arange(0, qubit_state_num))
I_q = np.identity(qubit_state_num)
H_q = np.diag(ens)

H0 = H_q

#Defining Concerned states (starting states)
g = np.array([1,0,0])
e = np.array([0,1,0])
psi0 = [g, e]

#Defining states to include in the drawing of occupation
states_draw_list = [0,1]
states_draw_names = ['0','1']

#Defining U (Target)
U = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])

#Defining U0 (Initial)
q_identity = np.identity(qubit_state_num**qubit_num)
U0 = q_identity

#Defining control Hs
XI = Q_x
YI = Q_y
ZI = Q_z
Hops = [XI, YI]
g_ops = 0.073036776 
ops_max_amp = [2 * np.pi * g_ops, 2 * np.pi * g_ops] # = Omega_{d, 0}
Hnames = ['x', 'y']

print(Hnames)

#Defining convergence parameters
max_iterations = 1000
decay = 5000 #max_iterations/2
convergence = {'rate': 0.1, 'update_step': 1, 'max_iterations': max_iterations, 'conv_target': 1e-6, 'learning_rate_decay': decay}

# guassian envelope pulse
reg_coeffs = {'envelope': 0.1, 'dwdt': 0.001}

['x', 'y']


In [10]:
def c_to_r_mat(M):
    # complex to real isomorphism for matrix
    return np.asarray(np.bmat([[M.real,-M.imag],[M.imag,M.real]]))

In [11]:
U = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])
c_to_r_mat(U)

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [8]:
U0

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
uks, U_final = Grape(H0, Hops, Hnames, U, total_time, steps, psi0,
                    convergence = convergence, 
                    draw = [states_draw_list, states_draw_names],
                    state_transfer = True,
                    use_gpu = False,
                    sparse_H = False,
                    show_plots = True, 
                    unitary_error = 1e-6, 
                    method = 'L-BFGS-B', 
                    maxA = ops_max_amp,
                    Taylor_terms = [20,0],
                    reg_coeffs = reg_coeffs,
                    save = True,
                    file_name = 'x01_lagos',
                    data_path = data_path)

Data saved at: ../output/x01_π/00077_x01_lagos.h5
Using 20 Taylor terms and 0 Scaling & Squaring terms
Building graph:
Propagators initialized.
self.sys_para.ops_len:  2
self.sys_para.ops_max_amp shape:  (2,)
self.ops_weight shape:  (2, 144)
self.ops_weight:  Tensor("weights:0", shape=(2, 144), dtype=float32)
Operators weight initialized.


ValueError: setting an array element with a sequence.